In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter

from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.cit import kci

from epc import epc

In [ ]:
import numpy as np

def generate_causal_chain_data(n):
    """
    Generate data based on a causal chain: A -> B -> C -> D -> E.

    Args:
        n (int): Number of samples.

    Returns:
        np.ndarray: A dataset with columns [A, B, C, D, E].
    """
    def random_nonlinear_function(x, function_type):
        """Applies a nonlinear transformation based on the specified type."""
        if function_type == "linear":
            return x
        elif function_type == "cubic":
            return x ** 3
        elif function_type == "tanh":
            return np.tanh(x)
        else:
            raise ValueError("Unsupported function type")

    # Define random nonlinear transformation types for each causal relationship
    F_type_A = np.random.choice(["linear", "cubic", "tanh"])
    F_type_B = np.random.choice(["linear", "cubic", "tanh"])
    F_type_C = np.random.choice(["linear", "cubic", "tanh"])
    F_type_D = np.random.choice(["linear", "cubic", "tanh"])
    F_type_E = np.random.choice(["linear", "cubic", "tanh"])

    # Generate noise for each variable
    E_A = np.random.normal(0, 1, n)
    E_B = np.random.normal(0, 1, n)
    E_C = np.random.normal(0, 1, n)
    E_D = np.random.normal(0, 1, n)
    E_E = np.random.normal(0, 1, n)

    # Generate data for each variable in the causal chain
    A = np.random.normal(0, 3, n)  # A is the root cause, no parents
    B = random_nonlinear_function(A + E_B, F_type_B)
    C = random_nonlinear_function(B + E_C, F_type_C)
    D = random_nonlinear_function(C + E_D, F_type_D)
    E = random_nonlinear_function(D + E_E, F_type_E)

    # Standardize each variable
    A = (A - np.mean(A)) / np.std(A)
    B = (B - np.mean(B)) / np.std(B)
    C = (C - np.mean(C)) / np.std(C)
    D = (D - np.mean(D)) / np.std(D)
    E = (E - np.mean(E)) / np.std(E)

    # Combine data into a single array
    data = np.array([A, B, C, D, E]).T

    return data


truSk = np.array([[ 0,  1,  0,  0,  0],  # A -> B
                  [-1,  0,  1,  0,  0],  # B -> C
                  [ 0, -1,  0,  1,  0],  # C -> D
                  [ 0,  0, -1,  0,  1],  # D -> E
                  [ 0,  0,  0, -1,  0]]) # No outgoing edge from E

In [ ]:
def gdata(n):
    
    Z = np.random.normal(0, 3, n)

    def random_nonlinear_function(x, function_type):
        if function_type == "linear":
            return x
        elif function_type == "cubic":
            return x**3
        elif function_type == "tanh":
            return np.tanh(x)
        else:
            raise ValueError("Unsupported function type")

    F_type = np.random.choice(["linear", "cubic", "tanh"])
    G_type = np.random.choice(["linear", "cubic", "tanh"])
    F_prime_type = np.random.choice(["linear", "cubic", "tanh"])
    G_prime_type = np.random.choice(["linear", "cubic", "tanh"])

    E_X = np.random.normal(0, 1, n)
    E_Y = np.random.normal(0, 1, n)

    X = random_nonlinear_function(random_nonlinear_function(Z, F_type) + E_X, G_type)
    meanX = np.mean(X)
    stdX = np.std(X)
    X = (X-meanX)/stdX

    Y = random_nonlinear_function(random_nonlinear_function(Z, F_prime_type) + E_Y, G_prime_type)
    meanY = np.mean(Y)
    stdY = np.std(Y)
    Y = (Y-meanY)/stdY

    data = np.array([X,Y,Z]).T

    return data
    """
    Z->X,Z-Y


    cg.G.graph[j,i]=1 and cg.G.graph[i,j]=-1 indicates  i --> j ,
    cg.G.graph[i,j] = cg.G.graph[j,i] = -1 indicates i --- j,
    cg.G.graph[i,j] = cg.G.graph[j,i] = 1 indicates i <-> j.
    """


truSk = np.array([[0, 0, 1],
                  [0, 0, 1],
                  [1, 1, 0]])

In [36]:
def tepc(data,k):
    re = []
    shuffled_data = data[np.random.permutation(len(data))]
    for sub_data in np.array_split(shuffled_data, k):
        cg = pc(sub_data, 0.01, kci, show_progress=False)
        sk = np.abs(cg.G.graph)
        re.append(sk)

    array_as_tuples = [(arr.shape, tuple(arr.flatten())) for arr in re]
    counter = Counter(array_as_tuples)
    most_common_tuple, count = counter.most_common(1)[0]
    most_common_array = np.array(most_common_tuple[1]).reshape(most_common_tuple[0])

    return most_common_array

In [58]:
t = 10
n = 400
err = 0


data = generate_causal_chain_data(n)
cg = pc(data, 0.01, kci)
sk = np.abs(cg.G.graph)

sk

  0%|          | 0/5 [00:00<?, ?it/s]

array([[0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0]])

In [54]:
t = 10
n = 400
err = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = generate_causal_chain_data(n)
    cg = pc(data, 0.01, kci, show_progress=False)
    sk = np.abs(cg.G.graph)
    if not np.all(np.abs(sk) == truSk):
        err += 1
err/t

Processing:   0%|          | 0/10 [00:04<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (5,5) (3,3) 

In [29]:
t = 100
n = 200
err = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    cg = pc(data, 0.01, kci, show_progress=False)
    sk =np.abs(cg.G.graph)
    if not np.all(np.abs(sk) == truSk):
        err += 1
err/t

Processing: 100%|██████████| 100/100 [00:13<00:00,  7.16it/s]


0.25

In [33]:
t = 100
n = 800
err = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    cg = pc(data, 0.01, kci, show_progress=False)
    sk =np.abs(cg.G.graph)
    if not np.all(np.abs(sk) == truSk):
        err += 1
err/t

Processing: 100%|██████████| 100/100 [03:12<00:00,  1.93s/it]


0.09

In [32]:
t = 100
n = 400
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "Gamma", 4, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing: 100%|██████████| 100/100 [00:36<00:00,  2.73it/s]


0.28

In [41]:
t = 100
n = 1600
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "ACAT", 4, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing: 100%|██████████| 100/100 [07:14<00:00,  4.34s/it]


0.12

In [40]:
t = 100
n = 1600
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    esk = tepc(data, 4)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing: 100%|██████████| 100/100 [03:36<00:00,  2.16s/it]


0.18

In [42]:
t = 100
n = 1600
err = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    cg = pc(data, 0.01, kci, show_progress=False)
    sk =np.abs(cg.G.graph)
    if not np.all(np.abs(sk) == truSk):
        err += 1
err/t

Processing: 100%|██████████| 100/100 [15:27<00:00,  9.28s/it]


0.1

In [43]:
t = 100
n = 1600
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "Gamma", 8, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing: 100%|██████████| 100/100 [03:27<00:00,  2.07s/it]


0.16

In [ ]:
t = 100
n = 1600
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "ACAT", 8, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing: 100%|██████████| 100/100 [13:43<00:00,  8.23s/it]


0.16

In [46]:
t = 1000
n = 3200
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "ACAT", 8, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing:  24%|██▍       | 241/1000 [36:03<1:53:33,  8.98s/it]


KeyboardInterrupt: 

In [45]:
t = 100
n = 3200
err = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    cg = pc(data, 0.01, kci, show_progress=False)
    sk =np.abs(cg.G.graph)
    if not np.all(np.abs(sk) == truSk):
        err += 1
err/t

Processing: 100%|██████████| 100/100 [1:57:41<00:00, 70.62s/it]


0.12